# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [1]:
# Load pickled data
import pickle
import numpy as np
import random

# TODO: Fill this in based on where you saved the training and testing data

training_file = 'train.p'
validation_file = 'valid.p'
testing_file = 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [2]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = X_train.shape[0]

# TODO: Number of validation examples
n_validation = X_valid.shape[0]

# TODO: Number of testing examples.
n_test = X_test.shape[0]

# TODO: What's the shape of an traffic sign image?
image_shape = (X_train.shape[1], X_train.shape[2])

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(set(y_train))

print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 34799
Number of validation examples = 4410
Number of testing examples = 12630
Image data shape = (32, 32)
Number of classes = 43


### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
import csv
from textwrap import wrap
# Visualizations will be shown in the notebook.
%matplotlib inline

# Load sign names
signnames = {}
with open('signnames.csv') as f:
    for sn in csv.DictReader(f):
        signnames[int(sn['ClassId'])] = sn['SignName']

def showImages(images, labels):
    fig = plt.figure(figsize=(50,60))
    for i, j in enumerate(random.sample(range(len(labels)), 100)):
        plt.subplot(10, 10, i+1)
        plt.title("\n".join(wrap(signnames[labels[j]], 20)), fontsize=20)
        plt.imshow(images[j])

In [ ]:
showImages(X_train, y_train)

In [ ]:
from collections import defaultdict
def showLabelDist(y_list, title):
    count = [0 for i in range(max(y_list) + 1)]
    labels = [i for i in range(max(y_list) + 1)]
    count_dict = defaultdict(int)
    for y in y_list:
        count[y] += 1
        count_dict[y] += 1
    plt.close('all')
    plt.title(title)
    plt.bar(labels, height=count)
    plt.show()
    
    for label in sorted(count_dict, key=lambda k: count_dict[k], reverse=True):
        print(signnames[label], ': ', count_dict[label])
    
    
showLabelDist(y_train, 'training')
showLabelDist(y_valid, 'validation')
showLabelDist(y_test, 'test')

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [3]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.
def normalize(X):
    X=X.astype(np.int16)
    mean = np.mean(X, axis=0)
    stddev = np.std(X, axis=0)
    return (X - mean) / stddev

X_train_norm = normalize(X_train)
X_valid_norm = normalize(X_valid)
X_test_norm = normalize(X_test)

### Model Architecture

In [4]:
### Define your architecture here.
### Feel free to use as many code cells as needed.
import tensorflow as tf
from tensorflow.contrib.layers import flatten
import time
from sklearn.utils import shuffle

class Model:
    def __init__(self, n_class):
        mu = 0
        sigma = 0.1
        
        self.X = tf.placeholder(tf.float32, shape=(None, 32, 32, 3))
        self.y = tf.placeholder(tf.int32, shape=(None))
        self.learn_rate = tf.placeholder(tf.float32)
        self.keep_prob = tf.placeholder(tf.float32)
        self.epoch = tf.Variable(0, name='epoch', trainable=False)
        
        #Conv layer 1
        conv1_depth = 32
        self.conv_W_1 = tf.Variable(tf.truncated_normal(shape=[3, 3, 3, conv1_depth], mean=mu, stddev=sigma), name='conv_W_1')
        self.conv_b_1 = tf.Variable(tf.zeros(conv1_depth), name='conv_b_1')
        conv1 = tf.nn.relu(tf.add(tf.nn.conv2d(self.X, self.conv_W_1, strides=[1, 1, 1, 1], padding='VALID'), self.conv_b_1))
        print(conv1.get_shape().as_list())
        conv1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID')
        print(conv1.get_shape().as_list())

        #Conv layer 2
        conv2_depth = 64
        self.conv_W_2 = tf.Variable(tf.truncated_normal(shape=[3, 3, conv1_depth, conv2_depth], mean=mu, stddev=sigma), name='conv_W_2')
        self.conv_b_2 = tf.Variable(tf.zeros(conv2_depth), name='conv_b_2')
        conv2 = tf.nn.relu(tf.add(tf.nn.conv2d(conv1, self.conv_W_2, strides=[1, 1, 1, 1], padding='VALID'), self.conv_b_2))
        print(conv2.get_shape().as_list())
        conv2 = tf.nn.max_pool(conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID')
        print(conv2.get_shape().as_list())

        fc_in = flatten(conv2)

        # Fully connected layers
        in_size = fc_in.get_shape().as_list()[1]
        print(in_size)
        fc1_size = 1024
        self.W_1 = tf.Variable(tf.truncated_normal(shape=[in_size, fc1_size], mean=mu, stddev=sigma), name='fc_W_1')
        self.b_1 = tf.Variable(tf.zeros(fc1_size), name='fc_b_1')
        fc1 = tf.nn.relu(tf.add(tf.matmul(fc_in, self.W_1), self.b_1))
        fc1 = tf.nn.dropout(fc1, self.keep_prob)
        
        fc2_size = 1024
        self.W_2 = tf.Variable(tf.truncated_normal(shape=[fc1_size, fc2_size], mean=mu, stddev=sigma), name='fc_W_2')
        self.b_2 = tf.Variable(tf.zeros(fc2_size), name='fc_b_2')
        fc2 = tf.nn.relu(tf.add(tf.matmul(fc1, self.W_2), self.b_2))
        fc2 = tf.nn.dropout(fc2, self.keep_prob)

        self.W_3 = tf.Variable(tf.truncated_normal(shape=[fc2_size, n_classes]), name='fc_W_3')
        self.b_3 = tf.Variable(tf.zeros(n_classes), name='fc_b_3')
        self.logits = tf.add(tf.matmul(fc2, self.W_3), self.b_3)

        # Training
        one_hot_y = tf.one_hot(self.y, n_classes)
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=one_hot_y))
        optimizer = tf.train.AdamOptimizer(learning_rate = self.learn_rate)
        self.train_op = optimizer.minimize(self.loss)
        self.increment_epoch_op = tf.assign_add(self.epoch, 1)

        # Validation
        # TODO: Per class accuracy, precision and recall
        correct = tf.equal(tf.argmax(self.logits, axis=1), tf.argmax(one_hot_y, axis=1))
        self.accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    def train(self, sess, X_train, y_train, learn_rate, keep_prob, batch_size=128):
        n = len(X_train)
        X_train, y_train = shuffle(X_train, y_train)
        losses = []
        for i in range(0, n, batch_size):
            batch_X = X_train[i:i+batch_size]
            batch_y = y_train[i:i+batch_size]
            _, batch_loss = sess.run([self.train_op, self.loss],
                                     feed_dict={self.X: batch_X, self.y: batch_y,
                                                self.learn_rate: learn_rate, self.keep_prob: keep_prob})
            losses.append(batch_loss)

        return sess.run(self.increment_epoch_op), np.mean(losses)

    def validate(self, sess, X_valid, y_valid, batch_size=4096):
        n = len(X_valid)
        n_correct = 0
        for i in range(0, n, batch_size):
            batch_X = X_valid[i:i+batch_size]
            batch_y = y_valid[i:i+batch_size]
            batch_accu = sess.run(self.accuracy, feed_dict={self.X: batch_X, self.y: batch_y, self.keep_prob: 1.0})
            print(batch_accu)
            n_correct += batch_accu * len(batch_X)
            
        return n_correct / n

def save(sess, path='ckpt/p2_model'):
    saver = tf.train.Saver()
    save_path = saver.save(sess, path)

    print('Session saved at %s' % save_path)

    return save_path

def restore(path='ckpt/p2_model'):
    sess = tf.Session()
    saver = tf.train.Saver()
    saver.restore(sess, path)

    print('Session restored from %s' % path)

    return sess

In [5]:
model = Model(n_classes)

[None, 30, 30, 32]
[None, 14, 14, 32]
[None, 12, 12, 64]
[None, 5, 5, 64]
1600


In [6]:
doRestore = True
if doRestore:
    sess = restore()
else:
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

Session restored from ckpt/p2_model


In [9]:
epoch=10
learn_rate = 0.0000001
keep_prob = 0.5
best = 0.929252

for ep in range(epoch):
    start = time.time()
    print('Epoch trained: %d. Loss: %f' % (model.train(sess, X_train_norm, y_train, learn_rate, keep_prob)))
    print('Took %f min' % ((time.time() - start) / 60))
    v = model.validate(sess, X_valid_norm, y_valid)
    print('Validation accuracy: %f' % v)
    if v > best:
        save(sess)
        best = v
#     print('Took totally %f min' % ((time.time() - start) / 60))

# save(sess)

Epoch trained: 67. Loss: 0.013813
Took 0.603398 min
0.92749
0.952229
Validation accuracy: 0.929252
Epoch trained: 68. Loss: 0.011976
Took 0.604311 min
0.92749
0.952229
Validation accuracy: 0.929252
Epoch trained: 69. Loss: 0.012437
Took 0.606029 min
0.92749
0.952229
Validation accuracy: 0.929252
Epoch trained: 70. Loss: 0.012525
Took 0.598007 min
0.92749
0.952229
Validation accuracy: 0.929252
Epoch trained: 71. Loss: 0.015500
Took 0.599429 min
0.92749
0.952229
Validation accuracy: 0.929252
Epoch trained: 72. Loss: 0.013895
Took 0.603272 min
0.92749
0.949045
Validation accuracy: 0.929025
Epoch trained: 73. Loss: 0.013920
Took 0.597944 min
0.92749
0.952229
Validation accuracy: 0.929252
Epoch trained: 74. Loss: 0.013059
Took 0.601575 min
0.92749
0.952229
Validation accuracy: 0.929252
Epoch trained: 75. Loss: 0.013176
Took 0.618617 min
0.92749
0.952229
Validation accuracy: 0.929252
Epoch trained: 76. Loss: 0.015471
Took 0.609786 min
0.927246
0.952229
Validation accuracy: 0.929025


In [ ]:
print('Training accuracy: %f' % model.validate(sess, X_train_norm, y_train))

In [7]:
print('Validation accuracy: %f' % model.validate(sess, X_valid_norm, y_valid))

0.927246
0.952229
Validation accuracy: 0.929025


In [ ]:
logits = sess.run(model.logits, feed_dict={model.X: X_train_norm[:10], model.y: y_train[:10]})
print(logits)

In [ ]:
print(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf.one_hot(y_train[:10], n_classes)).eval(session=sess))

In [ ]:
print(tf.one_hot(y_train[:10], n_classes).eval(session=sess))

In [ ]:
print(X_train.shape)
# print(model.W_1.eval(session=sess))
# print(m.epoch.eval(session=m.session))

In [ ]:
for v in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
    print(v)

In [ ]:
writer = tf.summary.FileWriter('logs', sess.graph)
print('Validation accuracy: %f' % model.validate(sess, X_train_norm[:5000], y_train[:5000]))
writer.close()

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")